In [0]:
%sql
with raw_data as (

  SELECT distinct
      X_device_platform as `platform_code`,  -- 필수 열
      X_device_product as `platform_version`,-- 필수 열
      X_Device_Country as `country_code`,    -- 필수/조건 열
      log_create_time,                       -- 필수/조건 열
      X_Device_SDK_VERSION as `dpv`,
      mac_addr,               -- 필수 열  
      x_device_sales_model as `sales_model_code`, -- 필수 열
      context_name,                          -- 필수/조건 열  
      message_id,                            -- 필수/조건 열
      normal_log:button_type as button_type
  FROM kic_data_ods.tlamp.normal_log_webos25
  WHERE 1=1
      AND X_device_country = 'KR'
      AND (context_name = 'lginput2' AND message_id = 'NL_BUTTON_CLICK')
      AND X_device_product = 'webOSTV 25'
      AND (x_device_sales_model LIKE '65QNED85AKS%' OR x_device_sales_model LIKE '75QNED85AKS%') -- EASY TV 모델
      AND date_ym >= '2025-10'
      AND date_ym < '2025-12'  

union all 

SELECT distinct
    X_device_platform as `platform_code`,  -- 필수 열
    X_device_product as `platform_version`,-- 필수 열
    X_Device_Country as `country_code`,    -- 필수/조건 열
    log_create_time,                       -- 필수/조건 열
    X_Device_SDK_VERSION as `dpv`,
    mac_addr,               -- 필수 열  
    x_device_sales_model as `sales_model_code`, -- 필수 열
    context_name,                          -- 필수/조건 열  
    message_id,                            -- 필수/조건 열
    normal_log:button_type as button_type
FROM kic_data_ods.tlamp.normal_log_webos25
WHERE 1=1
    AND X_device_country = 'KR'
    AND (context_name = 'LSM' AND message_id = 'NL_INPUT_KEYFILTER_USE')
    AND X_device_product = 'webOSTV 25'
    AND (x_device_sales_model LIKE '65QNED85AKS%' OR x_device_sales_model LIKE '75QNED85AKS%') -- EASY TV 모델
    AND date_ym >= '2025-10'
    AND date_ym < '2025-12' 
), fact_count_ud_by_buttons as (
    select button_type
        , count(mac_addr) as count_log
        , count(distinct mac_addr) as count_ud
    from   raw_data
    group by button_type
), fact_total_ud as (
    select count(distinct mac_addr) as total_ud
    from   raw_data
)

select * from fact_count_ud_by_buttons, fact_total_ud

In [0]:
%sql
with dim_mac_target as (

  SELECT distinct mac_addr
  FROM kic_data_ods.tlamp.normal_log_webos25
  WHERE 1=1
      AND X_device_country = 'KR'
      AND (context_name = 'lginput2' AND message_id = 'NL_BUTTON_CLICK')
      AND X_device_product = 'webOSTV 25'
      AND (x_device_sales_model LIKE '65QNED85AKS%' OR x_device_sales_model LIKE '75QNED85AKS%') -- EASY TV 모델
      AND date_ym >= '2025-10'
      AND date_ym < '2025-12'  
      AND normal_log:button_type in ('KEY_1', 'KEY_GUIDE')

  union all 

  SELECT distinct mac_addr
  FROM kic_data_ods.tlamp.normal_log_webos25
  WHERE 1=1
      AND X_device_country = 'KR'
      AND (context_name = 'LSM' AND message_id = 'NL_INPUT_KEYFILTER_USE')
      AND X_device_product = 'webOSTV 25'
      AND (x_device_sales_model LIKE '65QNED85AKS%' OR x_device_sales_model LIKE '75QNED85AKS%') -- EASY TV 모델
      AND date_ym >= '2025-10'
      AND date_ym < '2025-12'  
      AND normal_log:button_type in ('KEY_1', 'KEY_GUIDE')
), raw_data as (

  SELECT distinct
      X_device_platform as `platform_code`,  -- 필수 열
      X_device_product as `platform_version`,-- 필수 열
      X_Device_Country as `country_code`,    -- 필수/조건 열
      log_create_time,                       -- 필수/조건 열
      X_Device_SDK_VERSION as `dpv`,
      mac_addr,               -- 필수 열  
      x_device_sales_model as `sales_model_code`, -- 필수 열
      context_name,                          -- 필수/조건 열  
      message_id,                            -- 필수/조건 열
      normal_log:button_type as button_type
  FROM kic_data_ods.tlamp.normal_log_webos25
  WHERE 1=1
      AND X_device_country = 'KR'
      AND (context_name = 'lginput2' AND message_id = 'NL_BUTTON_CLICK')
      AND X_device_product = 'webOSTV 25'
      AND (x_device_sales_model LIKE '65QNED85AKS%' OR x_device_sales_model LIKE '75QNED85AKS%') -- EASY TV 모델
      AND date_ym >= '2025-10'
      AND date_ym < '2025-12'  


  union all 

  SELECT distinct
    X_device_platform as `platform_code`,  -- 필수 열
    X_device_product as `platform_version`,-- 필수 열
    X_Device_Country as `country_code`,    -- 필수/조건 열
    log_create_time,                       -- 필수/조건 열
    X_Device_SDK_VERSION as `dpv`,
    mac_addr,               -- 필수 열  
    x_device_sales_model as `sales_model_code`, -- 필수 열
    context_name,                          -- 필수/조건 열  
    message_id,                            -- 필수/조건 열
    normal_log:button_type as button_type
  FROM kic_data_ods.tlamp.normal_log_webos25
  WHERE 1=1
    AND X_device_country = 'KR'
    AND (context_name = 'LSM' AND message_id = 'NL_INPUT_KEYFILTER_USE')
    AND X_device_product = 'webOSTV 25'
    AND (x_device_sales_model LIKE '65QNED85AKS%' OR x_device_sales_model LIKE '75QNED85AKS%') -- EASY TV 모델
    AND date_ym >= '2025-10'
    AND date_ym < '2025-12' 
), mart_target_data as (
  select *
  from   raw_data 
  where  exists (select 1 from dim_mac_target where mac_addr = raw_data.mac_addr)
), fact_count_ud_by_buttons as (
    select button_type
        , count(mac_addr) as count_log
        , count(distinct mac_addr) as count_ud
    from   mart_target_data
    group by button_type
), fact_total_ud as (
    select count(distinct mac_addr) as total_ud
    from   mart_target_data
)

select * from fact_count_ud_by_buttons, fact_total_ud

-- 137, 137
-- select  (select count(distinct mac_addr)
--          from   raw_data
--          where  exists (select 1 from target_mac_addr where mac_addr = raw_data.mac_addr)) as matched_mac_count,
--         (select count(distinct mac_addr)
--          from target_mac_addr) as target_mac_count



In [0]:
%sql
with raw_data as (

  SELECT distinct
      X_device_platform as `platform_code`,  -- 필수 열
      X_device_product as `platform_version`,-- 필수 열
      X_Device_Country as `country_code`,    -- 필수/조건 열
      log_create_time,                       -- 필수/조건 열
      X_Device_SDK_VERSION as `dpv`,
      mac_addr,               -- 필수 열  
      x_device_sales_model as `sales_model_code`, -- 필수 열
      context_name,                          -- 필수/조건 열  
      message_id,                            -- 필수/조건 열
      normal_log:button_type as button_type
  FROM kic_data_ods.tlamp.normal_log_webos25
  WHERE 1=1
      AND X_device_country = 'KR'
      AND (context_name = 'lginput2' AND message_id = 'NL_BUTTON_CLICK')
      AND X_device_product = 'webOSTV 25'
      -- AND (x_device_sales_model LIKE '65QNED85AKS%' OR x_device_sales_model LIKE '75QNED85AKS%') -- EASY TV 모델
      AND date_ym >= '2025-10'
      AND date_ym < '2025-12'  

union all 

SELECT distinct
    X_device_platform as `platform_code`,  -- 필수 열
    X_device_product as `platform_version`,-- 필수 열
    X_Device_Country as `country_code`,    -- 필수/조건 열
    log_create_time,                       -- 필수/조건 열
    X_Device_SDK_VERSION as `dpv`,
    mac_addr,               -- 필수 열  
    x_device_sales_model as `sales_model_code`, -- 필수 열
    context_name,                          -- 필수/조건 열  
    message_id,                            -- 필수/조건 열
    normal_log:button_type as button_type
FROM kic_data_ods.tlamp.normal_log_webos25
WHERE 1=1
    AND X_device_country = 'KR'
    AND (context_name = 'LSM' AND message_id = 'NL_INPUT_KEYFILTER_USE')
    AND X_device_product = 'webOSTV 25'
    -- AND (x_device_sales_model LIKE '65QNED85AKS%' OR x_device_sales_model LIKE '75QNED85AKS%') -- EASY TV 모델
    AND date_ym >= '2025-10'
    AND date_ym < '2025-12' 
), fact_count_ud_by_buttons as (
    select button_type
        , count(mac_addr) as count_log
        , count(distinct mac_addr) as count_ud
    from   raw_data
    group by button_type
), fact_total_ud as (
    select count(distinct mac_addr) as total_ud
    from   raw_data
)

select * from fact_count_ud_by_buttons, fact_total_ud
